# 🔍 ดึงข้อมูลหุ้นกลุ่ม SET100 จาก Yahoo Finance + SET.or.th

In [ ]:
!pip install yfinance

In [ ]:
import requests
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm

# 1️⃣ ดึงรายชื่อหุ้นในกลุ่ม SET100 จากเว็บ SET.or.th

In [ ]:
set100_url = "https://www.set.or.th/en/market/index/set100"
res = requests.get(set100_url)
soup = BeautifulSoup(res.text, "html.parser")
# หา symbol จากตารางในหน้าเว็บ
symbols = []
for tag in soup.select("table.table-info tbody tr td:first-child"):
    sym = tag.text.strip()
    if sym:
        symbols.append(sym)

print(f"✅ พบหุ้นใน SET100 จำนวน {len(symbols)} ตัว")

# 2️⃣ สร้างฟังก์ชันดึงข้อมูลจาก Yahoo Finance

In [ ]:
def get_yf_data(symbol):
    try:
        ticker = yf.Ticker(f"{symbol}.BK")
        info = ticker.info
        return {
            "Symbol": symbol,
            "Market Cap": info.get("marketCap"),
            "P/E": info.get("trailingPE"),
            "P/BV": info.get("priceToBook"),
            "Dividend Yield (%)": (info.get("dividendYield") or 0) * 100,
            "EPS Growth (%)": (info.get("earningsGrowth") or 0) * 100,
            "Volume": info.get("volume"),
            "Turnover": (info.get("volume") or 0) * (info.get("currentPrice") or 0),
            "Sector": info.get("sector"),
            "Industry": info.get("industry"),
        }
    except Exception as e:
        return {"Symbol": symbol, "Error": str(e)}

# 3️⃣ ดึงข้อมูลทั้งหมด (ใช้ tqdm เพื่อแสดง progress)

In [ ]:
data = []
for sym in tqdm(symbols):
    data.append(get_yf_data(sym))
    sleep(1)  # หน่วงเล็กน้อยเพื่อไม่โดน block

df = pd.DataFrame(data)

# 4️⃣ ดึง Sector / Industry / Market Category จาก SET.or.th

In [ ]:
def get_set_info(symbol):
    try:
        url = f"https://www.set.or.th/en/market/product/stock/quote/{symbol}/company-profile"
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        info_dict = {"Symbol": symbol}

        # Sector / Industry
        info_table = soup.find("div", {"class": "row table-info"}).text
        if "Industry" in info_table:
            info_dict["Industry (SET)"] = soup.select_one("div:contains('Industry') + div").text.strip()
        if "Sector" in info_table:
            info_dict["Sector (SET)"] = soup.select_one("div:contains('Sector') + div").text.strip()

        # Market (SET / mai)
        market = soup.find("div", string=lambda t: t and "Market" in t)
        if market:
            info_dict["Market"] = market.find_next("div").text.strip()

        return info_dict
    except:
        return {"Symbol": symbol}

set_data = []
for sym in tqdm(symbols):
    set_data.append(get_set_info(sym))
    sleep(0.5)

df_set = pd.DataFrame(set_data)


# 5️⃣ รวมข้อมูลทั้งสองฝั่งเข้าด้วยกัน

In [ ]:
df_final = pd.merge(df, df_set, on="Symbol", how="left")

# 6️⃣ บันทึกเป็น CSV และ Excel

In [ ]:
df_final.to_csv("stocks_summary_SET100.csv", index=False, encoding="utf-8-sig")
df_final.to_excel("stocks_summary_SET100.xlsx", index=False)

print("🎯 บันทึกไฟล์แล้ว: stocks_summary_SET100.csv และ stocks_summary_SET100.xlsx")
print(df_final.head())

In [ ]:
# ======================================================
# 📊 ดึงข้อมูลหุ้นกลุ่ม SET100 จาก API ไม่เป็นทางการของ settrade.com
# ======================================================

import requests
import pandas as pd
from tqdm import tqdm
from time import sleep

In [ ]:
# ------------------------------------------------------
# 1️⃣ ดึงรายชื่อหุ้นใน SET100 จาก endpoint ของ settrade
# ------------------------------------------------------
url_set100 = "https://www.settrade.com/api/set/stock/market-realtime/index/info?indexId=SET100"

res = requests.get(url_set100)
data = res.json()

# ดึง symbol ทั้งหมดใน SET100
symbols = [i['symbol'] for i in data['indexConstituents']]
print(f"✅ พบหุ้นใน SET100 จำนวน {len(symbols)} ตัว")


In [ ]:
import requests
import pandas as pd

# ------------------------------------------------------
# ✅ ดึงรายชื่อหุ้นใน SET100 จาก API settrade พร้อม header
# ------------------------------------------------------
url_set100 = "https://www.settrade.com/api/set/stock/market-realtime/index/info?indexId=SET100"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://www.settrade.com/",
}

res = requests.get(url_set100, headers=headers)
print("Status code:", res.status_code)

# ตรวจสอบเนื้อหาที่ได้
if res.status_code == 200:
    try:
        data = res.json()
        symbols = [i["symbol"] for i in data["indexConstituents"]]
        print(f"✅ พบหุ้นใน SET100 จำนวน {len(symbols)} ตัว")
        print(symbols[:10])  # ดูตัวอย่าง 10 ตัวแรก
    except Exception as e:
        print("⚠️ ไม่สามารถอ่าน JSON ได้:", e)
        print("เนื้อหาที่ได้:", res.text[:300])
else:
    print("❌ โหลดข้อมูลไม่สำเร็จ:", res.status_code)
    print("เนื้อหาที่ได้:", res.text[:300])


In [ ]:
pip install cloudscraper


In [ ]:
import cloudscraper
import pandas as pd

# ------------------------------------------------------
# 1️⃣ ใช้ cloudscraper เพื่อ bypass firewall
# ------------------------------------------------------
scraper = cloudscraper.create_scraper()  # จำลอง browser

url_set100 = "https://www.settrade.com/api/set/stock/market-realtime/index/info?indexId=SET100"
res = scraper.get(url_set100)

print("Status code:", res.status_code)

if res.status_code == 200:
    data = res.json()
    symbols = [i["symbol"] for i in data["indexConstituents"]]
    print(f"✅ พบหุ้นใน SET100 จำนวน {len(symbols)} ตัว")
    print(symbols[:10])
else:
    print("❌ โหลดข้อมูลไม่สำเร็จ:", res.status_code)
    print(res.text[:300])


-------------------------------------------------------------
ทดลอง 3

📊 ดึงข้อมูลหุ้น SET100 พร้อมตัวชี้วัดทางการเงินจาก settrade.com


In [15]:
import cloudscraper
import pandas as pd
import time

In [16]:
# 1️⃣ สร้าง scraper เพื่อเลี่ยง firewall (Incapsula)
scraper = cloudscraper.create_scraper()

In [17]:
# 2️⃣ ดึงรายชื่อหุ้นใน SET100
url_set100 = "https://www.settrade.com/api/set/stock/market-realtime/index/info?indexId=SET100"
res = scraper.get(url_set100)
data = res.json()

symbols = [i["symbol"] for i in data["indexConstituents"]]
print(f"✅ พบหุ้นใน SET100 จำนวน {len(symbols)} ตัว\n")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
print("Status code:", res.status_code)
print(res.text[:500])


Status code: 403
<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial-scale=1.0"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><script type="text/javascript" src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3"></script></head><body style="margin:0px;height:100%"><iframe id="main-iframe" src="/_Incapsula_Resource?CWUDNSAI=23&xinfo=48-187524564-0%202NNN%20RT%281759


------------------

In [21]:
import cloudscraper
import pandas as pd
from time import sleep
from tqdm import tqdm

# สร้าง scraper
scraper = cloudscraper.create_scraper()

# 1️⃣ ดึงรายชื่อหุ้น SET100
url_set100 = "https://www.set.or.th/api/set/stock/list?language=th&market=SET"
res = scraper.get(url_set100)
print("Status code:", res.status_code)

if res.status_code != 200:
    raise Exception(f"โหลดข้อมูลไม่สำเร็จ: {res.status_code}")

data = res.json()

# ดึงเฉพาะ SET100
set100_symbols = [item["symbol"] for item in data["securitySymbols"] if item.get("indexType") == "SET100"]
print(f"✅ พบหุ้นใน SET100 จำนวน {len(set100_symbols)} ตัว")

# 2️⃣ ฟังก์ชันดึงข้อมูลหุ้นแต่ละตัว
def get_stock_info(symbol):
    url_info = f"https://www.set.or.th/api/set/company/{symbol}?lang=th"
    try:
        r = scraper.get(url_info)
        r.raise_for_status()
        info = r.json()["securityInfo"]
        return {
            "Symbol": symbol,
            "CompanyName": info.get("companyName"),
            "Market": info.get("marketName"),
            "Sector": info.get("sectorName"),
            "Industry": info.get("industryName"),
            "MarketCap": info.get("marketCap"),
            "PE": info.get("peRatio"),
            "PBV": info.get("pbvRatio"),
            "DividendYield": info.get("dividendYield"),
            "EPS": info.get("eps"),
            "Volume": info.get("averageVolume"),
            "Turnover": info.get("averageTurnover"),
        }
    except Exception as e:
        print(f"⚠️ Error fetching {symbol}: {e}")
        return None

# 3️⃣ ดึงข้อมูลทุกหุ้น SET100
all_data = []
for sym in tqdm(set100_symbols):
    stock_info = get_stock_info(sym)
    if stock_info:
        all_data.append(stock_info)
    sleep(0.2)

# 4️⃣ แปลงเป็น DataFrame
df = pd.DataFrame(all_data)

# 5️⃣ จัดเรียงคอลัมน์
df = df[
    ["Symbol", "CompanyName", "Market", "Sector", "Industry",
     "MarketCap", "PE", "PBV", "DividendYield", "EPS",
     "Volume", "Turnover"]
]

# 6️⃣ แสดงตัวอย่าง
display(df.head(10))

# 7️⃣ บันทึก Excel
df.to_excel("SET100_Data.xlsx", index=False)
print("💾 บันทึกไฟล์เรียบร้อย: SET100_Data.xlsx")


Status code: 403


Exception: โหลดข้อมูลไม่สำเร็จ: 403

---------------------------

In [23]:
pip install settrade-v2


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=65647 sha256=0ac5b22455b93b93d724ce3451b2be1faf738aeffb0667949692e32fe6d47f42
  Stored in directory: c:\users\a0864\appdata\local\pip\cache\wheels\f9\58\b1\af83cc69fc3ea9dcfafeb61e26c7fa7bcfca4fdf276791c6dc
  Created wheel for betterproto: filename=betterproto-1.2.5-py3-none-any.whl size=22109 sha256=058d2c2407558a92a6696593b361e3690a4cf6652fbf646b14138249edb0d18d
  Stored in directory

  DEPRECATION: Building 'paho-mqtt' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'paho-mqtt'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'stringcase' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'stringcase'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [24]:
# ติดตั้ง SDK
# pip install settrade-v2

from settrade_v2 import Investor
from settrade_v2.errors import SettradeError
import pandas as pd

# -------------------------------
# 1️⃣ ตั้งค่าคีย์และบัญชี
# -------------------------------
APP_ID = "gjwwGfOcEibaEgst"
APP_SECRET = "ALKtlU4wod9PW3RJY8i/Cs9M2F9s9K/gcl9+B18NMNc4"
BROKER_ID = "SANDBOX"
APP_CODE = "SANDBOX"
ACCOUNT_NO = "ami-E"

# -------------------------------
# 2️⃣ สร้าง Investor object
# -------------------------------
investor = Investor(
    app_id=APP_ID,
    app_secret=APP_SECRET,
    broker_id=BROKER_ID,
    app_code=APP_CODE,
    is_auto_queue=False
)

# -------------------------------
# 3️⃣ ดึงหุ้นใน SET100
# -------------------------------
# สมมติ Market API ของ SDK มี method get_set100() หรือ get_index_components()
# คุณต้องเช็ก spec ของ SDK Broker ของคุณ
try:
    set100_symbols = investor.MarketData().get_index_components(index_id="SET100")  # ตัวอย่าง
    symbols = [s["symbol"] for s in set100_symbols]
    print(f"✅ SET100 symbols: {symbols}")
except SettradeError as e:
    print("❌ ดึง SET100 ล้มเหลว:", e)
    symbols = []

# -------------------------------
# 4️⃣ ดึงข้อมูล Market Data ของแต่ละหุ้น
# -------------------------------
all_data = []
for sym in symbols:
    try:
        stock_info = investor.MarketData().get_stock_info(symbol=sym)  # ตัวอย่าง
        all_data.append({
            "Symbol": sym,
            "CompanyName": stock_info.get("companyName"),
            "Market": stock_info.get("marketName"),
            "Sector": stock_info.get("sectorName"),
            "Industry": stock_info.get("industryName"),
            "MarketCap": stock_info.get("marketCap"),
            "PE": stock_info.get("peRatio"),
            "PBV": stock_info.get("pbvRatio"),
            "DividendYield": stock_info.get("dividendYield"),
            "EPS": stock_info.get("eps"),
            "Volume": stock_info.get("averageVolume"),
            "Turnover": stock_info.get("averageTurnover"),
        })
    except SettradeError as e:
        print(f"⚠️ ดึงข้อมูล {sym} ล้มเหลว:", e)

# -------------------------------
# 5️⃣ แปลงเป็น DataFrame และบันทึก
# -------------------------------
df = pd.DataFrame(all_data)
df.to_excel("SET100_MarketData.xlsx", index=False)
print("💾 บันทึกไฟล์เรียบร้อย: SET100_MarketData.xlsx")


AttributeError: 'MarketData' object has no attribute 'get_index_components'

In [25]:
from settrade_v2 import Investor
from settrade_v2.errors import SettradeError
import pandas as pd

# -------------------------------
# 1️⃣ ตั้งค่า Sandbox
# -------------------------------
APP_ID = "gjwwGfOcEibaEgst"       # ตัวอย่างจากคุณ
APP_SECRET = "ALKtlU4wod9PW3RJY8i/Cs9M2F9s9K/gcl9+B18NMNc4"
BROKER_ID = "SANDBOX"
APP_CODE = "SANDBOX"
EQUITY_ACCOUNT = "ami-E"
DERI_ACCOUNT = "ami-D"
PIN = "000000"

# -------------------------------
# 2️⃣ สร้าง Investor object
# -------------------------------
investor = Investor(
    app_id=APP_ID,
    app_secret=APP_SECRET,
    broker_id=BROKER_ID,
    app_code=APP_CODE,
    is_auto_queue=False
)

# -------------------------------
# 3️⃣ ดึงข้อมูลบัญชี Equity Sandbox
# -------------------------------
equity = investor.Equity(account_no=EQUITY_ACCOUNT)
try:
    account_info = equity.get_account_info()
    print("✅ ข้อมูลบัญชี Equity Sandbox:")
    print(account_info)
except SettradeError as e:
    print("❌ ดึงข้อมูลบัญชีล้มเหลว:", e)

# -------------------------------
# 4️⃣ ดึง Market Data ตัวอย่าง (Sandbox)
# -------------------------------
symbols = ["PTT", "AOT", "SCB"]  # ตัวอย่าง
all_data = []

for sym in symbols:
    try:
        stock_info = investor.MarketData().get_stock_info(symbol=sym)
        all_data.append({
            "Symbol": sym,
            "CompanyName": stock_info.get("companyName"),
            "Market": stock_info.get("marketName"),
            "Sector": stock_info.get("sectorName"),
            "Industry": stock_info.get("industryName"),
            "MarketCap": stock_info.get("marketCap"),
            "PE": stock_info.get("peRatio"),
            "PBV": stock_info.get("pbvRatio"),
            "DividendYield": stock_info.get("dividendYield"),
            "EPS": stock_info.get("eps"),
            "Volume": stock_info.get("averageVolume"),
            "Turnover": stock_info.get("averageTurnover"),
        })
    except SettradeError as e:
        print(f"⚠️ ดึงข้อมูล {sym} ล้มเหลว:", e)

# แปลงเป็น DataFrame
df = pd.DataFrame(all_data)
display(df)


✅ ข้อมูลบัญชี Equity Sandbox:
{'lineAvailable': 10000000.0, 'creditLimit': 10000000.0, 'cashBalance': 10000000.0, 'accountType': 'CASH_ACCOUNT', 'clientType': 'INDIVIDUAL', 'customerType': 'CUSTOMER', 'canBuy': True, 'canSell': True, 'crossingKey': '0', 'initialCreditLimit': 10000000.0, 'initialCashBalance': 10000000.0, 'initialLineAvailable': 10000000.0, 'creditBalance': False}


AttributeError: 'MarketData' object has no attribute 'get_stock_info'

----------------

In [27]:
pip install selenium

  Using cached selenium-4.36.0-py3-none-any.whl.metadata (7.5 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.6 MB 7.6 MB/s eta 0:00:02
   ---------------- ----------------------- 3.9/9.6 MB 13.9 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.6 MB 17.1 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 16.3 MB/s eta 0:00:00

  Attempting uninstall: urllib3

    Found existing installation: urllib3 2.3.0

    Uninstalling urllib3-2.3.0:

      Successfully uninstalled urllib3-2.3.0

   ----- ---------------------------------- 1/8 [urllib3]
  Attempting uninstall: typing_extensions
   ----- ---------------------------------- 1/8 [urllib3]
    Found existing installation: typing_extensions 4.12.2
   ----- ---------------------------------- 1/8 [urllib3]
    Uninstalling typing_extensions-4.12.2:
   ----- ---------------------------------- 1/8 [urllib3]
   ---------- --------

In [28]:
pip install selenium pandas openpyxl tqdm


Note: you may need to restart the kernel to use updated packages.


In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm
import time

# -------------------------------
# 1️⃣ ตั้งค่า Chrome Driver
# -------------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")  # รันแบบไม่มีหน้าต่าง
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

service = Service("D:\chromedriver-win64\chromedriver.exe")  # <-- ใส่ path ของ chromedriver

driver = webdriver.Chrome(service=service, options=chrome_options)

# -------------------------------
# 2️⃣ เปิดหน้า SET100
# -------------------------------
url = "https://www.set.or.th/set100/company-SET100.html"  # ตัวอย่างหน้า SET100
driver.get(url)
time.sleep(3)  # รอโหลดหน้าเว็บ

# -------------------------------
# 3️⃣ ดึงรายชื่อหุ้นและลิงก์
# -------------------------------
rows = driver.find_elements(By.CSS_SELECTOR, "table#companyTable tbody tr")
symbols = []
links = []

for row in rows:
    try:
        cells = row.find_elements(By.TAG_NAME, "td")
        sym = cells[1].text.strip()  # Symbol
        link = cells[1].find_element(By.TAG_NAME, "a").get_attribute("href")
        symbols.append(sym)
        links.append(link)
    except:
        continue

print(f"✅ พบหุ้นใน SET100 จำนวน {len(symbols)} ตัว")

# -------------------------------
# 4️⃣ ดึงข้อมูลหุ้นแต่ละตัว
# -------------------------------
all_data = []

for sym, link in tqdm(zip(symbols, links), total=len(symbols)):
    driver.get(link)
    time.sleep(2)  # รอโหลดหน้า

    try:
        data = {
            "Symbol": sym,
            "CompanyName": driver.find_element(By.CSS_SELECTOR, "h1.company-name").text,
            "Market": driver.find_element(By.CSS_SELECTOR, "div.company-info span.market").text,
            "Sector": driver.find_element(By.XPATH, "//th[text()='Sector']/following-sibling::td").text,
            "Industry": driver.find_element(By.XPATH, "//th[text()='Industry']/following-sibling::td").text,
            "MarketCap": driver.find_element(By.XPATH, "//th[text()='Market Cap']/following-sibling::td").text,
            "PE": driver.find_element(By.XPATH, "//th[text()='P/E']/following-sibling::td").text,
            "PBV": driver.find_element(By.XPATH, "//th[text()='P/BV']/following-sibling::td").text,
            "DividendYield": driver.find_element(By.XPATH, "//th[text()='Dividend Yield']/following-sibling::td").text,
            "EPS": driver.find_element(By.XPATH, "//th[text()='EPS']/following-sibling::td").text,
            "Volume": driver.find_element(By.XPATH, "//th[text()='Volume']/following-sibling::td").text,
            "Turnover": driver.find_element(By.XPATH, "//th[text()='Turnover']/following-sibling::td").text,
        }
        all_data.append(data)
    except:
        print(f"⚠️ ดึงข้อมูล {sym} ไม่สำเร็จ")

# -------------------------------
# 5️⃣ แปลงเป็น DataFrame
# -------------------------------
df = pd.DataFrame(all_data)

# -------------------------------
# 6️⃣ Export Excel
# -------------------------------
df.to_excel("SET100_Data_Selenium.xlsx", index=False)
print("💾 บันทึกไฟล์เรียบร้อย: SET100_Data_Selenium.xlsx")

# -------------------------------
# 7️⃣ ปิด Browser
# -------------------------------
driver.quit()


✅ พบหุ้นใน SET100 จำนวน 0 ตัว


0it [00:00, ?it/s]


💾 บันทึกไฟล์เรียบร้อย: SET100_Data_Selenium.xlsx


-----------------

In [34]:
import requests
import pandas as pd

# -------------------------------
# 1️⃣ ดึงข้อมูลหุ้นทั้งหมดจาก SET API
# -------------------------------
url = "https://www.set.or.th/api/set/stock/list?language=th&market=SET"
res = requests.get(url)
res.raise_for_status()  # ตรวจสอบการโหลด
data = res.json()

# -------------------------------
# 2️⃣ ดึงเฉพาะ SET100
# -------------------------------
set100 = [d for d in data if "SET100" in d.get("index", "")]

# -------------------------------
# 3️⃣ ฟังก์ชันแปลงตัวเลข
# -------------------------------
def parse_number(s):
    """แปลง string เป็น float ถ้ามี ',' หรือ '%'"""
    if s is None:
        return None
    s = str(s).replace(",", "").replace("%", "").strip()
    try:
        return float(s)
    except:
        return None

# -------------------------------
# 4️⃣ สร้าง DataFrame
# -------------------------------
all_data = []
for stock in set100:
    all_data.append({
        "Symbol": stock.get("symbol"),
        "CompanyName": stock.get("companyName"),
        "Market": stock.get("market"),
        "Sector": stock.get("sector"),
        "Industry": stock.get("industry"),
        "MarketCap": parse_number(stock.get("marketCap")),
        "PE": parse_number(stock.get("peRatio")),
        "PBV": parse_number(stock.get("pbvRatio")),
        "DividendYield": parse_number(stock.get("dividendYield")),
        "EPS": parse_number(stock.get("eps")),
        "Volume": parse_number(stock.get("volume")),
        "Turnover": parse_number(stock.get("turnover")),
    })

df = pd.DataFrame(all_data)

# -------------------------------
# 5️⃣ Export Excel
# -------------------------------
df.to_excel("SET100_Data_API_Complete.xlsx", index=False)
print("💾 บันทึกไฟล์เรียบร้อย: SET100_Data_API_Complete.xlsx")


HTTPError: 403 Client Error: Forbidden for url: https://www.set.or.th/api/set/stock/list?language=th&market=SET

-----------

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from tqdm import tqdm
import re

# -------------------------------
# 1️⃣ ตั้งค่า ChromeDriver
# -------------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

service = Service(r"D:\chromedriver-win64\chromedriver.exe")  # <-- ใส่ path ของคุณ
driver = webdriver.Chrome(service=service, options=chrome_options)

# -------------------------------
# 2️⃣ เปิดหน้า SET100
# -------------------------------
url = "https://www.set.or.th/set100/company-SET100.html"
driver.get(url)
wait = WebDriverWait(driver, 15)

# -------------------------------
# 3️⃣ รอให้ table โหลด (JS-rendered)
# -------------------------------
# Table ของ SET100 จะอยู่ใน class "table table-hover table-condensed"
time.sleep(3)  # รอ JavaScript render
rows = driver.find_elements(By.CSS_SELECTOR, "table.table-hover tbody tr")
print(f"✅ พบหุ้นใน SET100 จำนวน {len(rows)} ตัว")

# -------------------------------
# 4️⃣ ดึง Symbol + Link
# -------------------------------
symbols = []
links = []

for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    sym = cells[1].text.strip()
    link = cells[1].find_element(By.TAG_NAME, "a").get_attribute("href")
    symbols.append(sym)
    links.append(link)

# -------------------------------
# 5️⃣ ฟังก์ชันช่วยแปลงตัวเลข
# -------------------------------
def parse_number(s):
    if s is None:
        return None
    s = str(s).replace(",", "").replace("%", "").strip()
    try:
        return float(s)
    except:
        return None

# -------------------------------
# 6️⃣ ดึงข้อมูลหุ้นแต่ละตัว
# -------------------------------
all_data = []

for sym, link in tqdm(zip(symbols, links), total=len(symbols)):
    driver.get(link)
    time.sleep(2)  # รอ JavaScript render
    try:
        data = {
            "Symbol": sym,
            "CompanyName": driver.find_element(By.CSS_SELECTOR, "h1.company-name").text,
            "Market": driver.find_element(By.XPATH, "//th[text()='Market']/following-sibling::td").text,
            "Sector": driver.find_element(By.XPATH, "//th[text()='Sector']/following-sibling::td").text,
            "Industry": driver.find_element(By.XPATH, "//th[text()='Industry']/following-sibling::td").text,
            "MarketCap": parse_number(driver.find_element(By.XPATH, "//th[text()='Market Cap']/following-sibling::td").text),
            "PE": parse_number(driver.find_element(By.XPATH, "//th[text()='P/E']/following-sibling::td").text),
            "PBV": parse_number(driver.find_element(By.XPATH, "//th[text()='P/BV']/following-sibling::td").text),
            "DividendYield": parse_number(driver.find_element(By.XPATH, "//th[text()='Dividend Yield']/following-sibling::td").text),
            "EPS": parse_number(driver.find_element(By.XPATH, "//th[text()='EPS']/following-sibling::td").text),
            "Volume": parse_number(driver.find_element(By.XPATH, "//th[text()='Volume']/following-sibling::td").text),
            "Turnover": parse_number(driver.find_element(By.XPATH, "//th[text()='Turnover']/following-sibling::td").text),
        }
        all_data.append(data)
    except Exception as e:
        print(f"⚠️ ดึงข้อมูล {sym} ไม่สำเร็จ:", e)

# -------------------------------
# 7️⃣ แปลงเป็น DataFrame + Export Excel
# -------------------------------
df = pd.DataFrame(all_data)
df.to_excel("SET100_Data_Selenium_Complete.xlsx", index=False)
print("💾 บันทึกไฟล์เรียบร้อย: SET100_Data_Selenium_Complete.xlsx")

# -------------------------------
# 8️⃣ ปิด Browser
# -------------------------------
driver.quit()


✅ พบหุ้นใน SET100 จำนวน 0 ตัว


0it [00:00, ?it/s]

💾 บันทึกไฟล์เรียบร้อย: SET100_Data_Selenium_Complete.xlsx
